## TASK I: CRRs - Most Trading
#### Yijia Zeng
---

### Task Objectives:
1. Collect the monthly/annual auction price data for CRRs in ERCOT at this link:
https://www.ercot.com/mp/data-products/data-product-details?id=NP7-803-M

2. Identify the CRRs with the most trading activities/volumes based on data in 1.
    - CRRs with highest total bid volume
    - CRRs with highest number of bids
    - CRR actual quantities awarded

3. Collect the Dayahead electricity prices associated with the most actively traded
CRRs found in 2.
     - DAM hourly LMP https://www.ercot.com/mp/data-products/data-product-details?id=NP4-183-CD -> calculate FTR actual payoff

4. Compare the auction prices and the realized payoffs of the CRRs found in 2.
    - Monthly Payout Distribution v.s. Auction Price Distribution
    - Average of Monthly Payout Distribution v.s. Average Auction Price Distribution
---


### Data 
- Monthly (Nov 2023 - Oct 2024)
    - Common_AuctionawardsAndOffers
        - Description: Corresponding list of CRR bids and offers for that particular market, including Source, Sink, Bid Type, Start Date, End Date, Hedge Type, Time-of-Use, MW, award Price and Shadow Price.
    - Common_MarketResults （？）
        - Description: orresponding list of CRR  results for that particular market and the AH, including CRR ID, Original CRR ID, Account Holder, Hedge Type, Class, CRR Type, Source, Sink, Start Date, End Date, Time-of-Use, award24Hour, MW, and Shadow Price.  
    - LMP in Dayahead Market
        - https://www.ercot.com/mp/data-products/data-product-details?id=NP4-183-CD
--- 

### Definition
- Most Trading Activities can be defined as
    - CRRs with highest total award volume
    - CRRs with highest number of awards
    - CRRs actual quantities awarded
---

#### Resources
1. CRRs Description https://www.ercot.com/files/docs/2020/05/12/2019_09_Set301_M2_-_CRR.pdf
    - OffPeak: 0100 - 0600; 2200 - 0000
    - PeakWE: 0700 - 2200
    - PeakWD: 0700 - 2200
2. DAM hourly LMP https://www.ercot.com/mp/data-products/data-product-details?id=NP4-183-CD -> calculate FTR actual payoff
3. Not-to-Exceed Price



In [198]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

In [199]:
CRR_bid = pd.read_csv('data/Common_AuctionBidsAndOffers_2023.NOV.Monthly.Auction_AUCTION.csv')
CRR_award = pd.read_csv('data/Common_MarketResults_2023.NOV.Monthly.Auction_AUCTION.csv')

### CRR Obligations - monthly
---
#### Workflow
- Filter all CRR obligation

- Seperating Data into Buy v.s. Sell

- Generate corresponding CRR with most trading/activities events identified by (source, sink, time of use)
    - Apply previous 3 evaluation standards
    - Select first 500 CRRs for each criteria

- Find related auction clearing price
    - Shadow price in Market Results

- Find Corresponding LMPs to calculate CRR payout
    - $$\sum (P_{sink} - P_{source})$$

In [200]:
CRR_bid_buy_obl = CRR_bid[(CRR_bid['BidType'] == "BUY") & (CRR_bid['HedgeType'] == "OBL")]
CRR_bid_sell_obl = CRR_bid[(CRR_bid['BidType'] == "SELL") &(CRR_bid['HedgeType'] == "OBL")]

In [201]:
first_k_highest = 1000

##### Find CRRs Obligations(Source, Sink, TimeOfUse) with most trading activities
---

In [202]:
CRR_bid_buy_highest_volume = (CRR_bid_buy_obl.groupby(['Source', 'Sink', 'TimeOfUse'], as_index=False)['MW']
                              .sum()
                              .sort_values('MW', ascending = False)
                              .reset_index(drop = True))[:first_k_highest]
CRR_bid_sell_highest_volume = (CRR_bid_sell_obl.groupby(['Source', 'Sink', 'TimeOfUse'], as_index=False)['MW']
                              .sum()
                              .sort_values('MW', ascending = False)
                              .reset_index(drop = True))[:first_k_highest]
CRR_bid_buy_highest_bids = (CRR_bid_buy_obl.groupby(['Source', 'Sink', 'TimeOfUse'], as_index=False)['MW']
                              .count()
                              .sort_values('MW', ascending = False)
                              .reset_index(drop = True))[:first_k_highest]
CRR_bid_sell_highest_bids = (CRR_bid_sell_obl.groupby(['Source', 'Sink', 'TimeOfUse'], as_index=False)['MW']
                              .count()
                              .sort_values('MW', ascending = False)
                              .reset_index(drop = True))[:first_k_highest]

In [203]:
CRR_award_buy_obl = CRR_award[(CRR_award['BidType'] == "BUY") & (CRR_award['HedgeType'] == "OBL")]
CRR_award_sell_obl = CRR_award[(CRR_award['BidType'] == "SELL") & (CRR_award['HedgeType'] == "OBL")]

In [204]:
CRR_award_buy_highest_volume = (CRR_award_buy_obl.groupby(['Source', 'Sink', 'TimeOfUse'], as_index=False)['MW']
                              .sum()
                              .sort_values('MW', ascending = False)
                              .reset_index(drop = True))[:first_k_highest]
CRR_award_sell_highest_volume = (CRR_award_sell_obl.groupby(['Source', 'Sink', 'TimeOfUse'], as_index=False)['MW']
                              .sum()
                              .sort_values('MW', ascending = False)
                              .reset_index(drop = True))[:first_k_highest]

##### Find Corresponding Auction Prices 
---
- Quantity * Auction Clearing Price

In [205]:
def find_price(market_result, df, identified_col, target_col):
    market_result = market_result[identified_col + target_col]
    df = pd.merge(df, market_result, on = identified_col, how = 'left')
    df['auction_total'] = df['MW'] * df[target_col[0]]
    return df

In [206]:
identified_col = ['Source', 'Sink', 'TimeOfUse']
target_col = ['ShadowPricePerMWH']
CRR_clearing_price_buy = CRR_award[CRR_award['BidType'] == "BUY"].groupby(identified_col, as_index=False)[target_col[0]].mean()
CRR_clearing_price_sell = CRR_award[CRR_award['BidType'] == "SELL"].groupby(identified_col, as_index=False)[target_col[0]].mean()

In [207]:
CRR_award_buy_highest_volume = find_price(CRR_clearing_price_buy, CRR_award_buy_highest_volume, identified_col, target_col)
CRR_bid_buy_highest_volume = find_price(CRR_clearing_price_buy, CRR_bid_buy_highest_volume, identified_col, target_col)
CRR_bid_buy_highest_bids = find_price(CRR_clearing_price_buy, CRR_bid_buy_highest_bids, identified_col, target_col)
CRR_award_sell_highest_volume = find_price(CRR_clearing_price_sell, CRR_award_sell_highest_volume, identified_col, target_col)
CRR_bid_sell_highest_volume = find_price(CRR_clearing_price_sell, CRR_bid_sell_highest_volume, identified_col, target_col)
CRR_bid_sell_highest_bids = find_price(CRR_clearing_price_sell, CRR_bid_sell_highest_bids, identified_col, target_col)

##### Find Actual Payoff of CRR
---
- 